In [ ]:
import pandas as pd
import recordlinkage

# 1) Caricamento e pre-elaborazione dati
schema_file = "main_outputs/final_mediated_schema.csv"
df = pd.read_csv(schema_file)

# Seleziona alcune colonne di interesse
df = df[['company_name', 'industry', 'headquarters_country', 'headquarters_city', 'year_founded']]
df.dropna(subset=['company_name'], inplace=True)

print(f"[Script con blocking combinato] Numero di record iniziali: {len(df)}")

# 2) Creazione di due insiemi di coppie candidate
#    A) Blocco su headquarters_country + SortedNeighbourhood su company_name
indexer_1 = recordlinkage.Index()
indexer_1.add(recordlinkage.index.Block('headquarters_country'))
indexer_1.add(recordlinkage.index.SortedNeighbourhood('company_name', window=5))
candidate_pairs_country = indexer_1.index(df)

#    B) Blocco su headquarters_city + SortedNeighbourhood su company_name
indexer_2 = recordlinkage.Index()
indexer_2.add(recordlinkage.index.Block('headquarters_city'))
indexer_2.add(recordlinkage.index.SortedNeighbourhood('company_name', window=5))
candidate_pairs_city = indexer_2.index(df)

# 3) Creazione delle coppie finali come UNIONE dei due insiemi
candidates_country_set = set(candidate_pairs_country)
candidates_city_set = set(candidate_pairs_city)
candidate_pairs = candidates_country_set.union(candidates_city_set)

print(f"[Script con blocking combinato] Numero di coppie candidate: {len(candidate_pairs)}")

# 4) Definizione delle regole di confronto
compare = recordlinkage.Compare()

# Similarità sul company_name (jarowinkler)
compare.string('company_name',
               'company_name',
               method='jarowinkler',
               label='name_sim')

# Similarità su industry (jarowinkler)
compare.string('industry',
               'industry',
               method='jarowinkler',
               label='industry_sim')

# Similarità su headquarters_country (jarowinkler)
compare.string('headquarters_country',
               'headquarters_country',
               method='jarowinkler',
               label='country_sim')

# Se desideri, puoi anche aggiungere la similarità su 'headquarters_city':
# compare.string('headquarters_city', 'headquarters_city', method='jarowinkler', label='city_sim')

# 5) Calcolo della matrice di similarità
#    (convertiamo il set di coppie in una lista, così `compare.compute()` non dà errori)
# ...
# 5) Calcolo della matrice di similarità
#    Convertiamo l'insieme di tuple in una lista e poi in un MultiIndex

candidate_pairs_list = list(candidate_pairs)

# Crea un MultiIndex a partire dalla lista di tuple
candidate_pairs_mi = pd.MultiIndex.from_tuples(candidate_pairs_list, names=['level_0', 'level_1'])

similarity_matrix = compare.compute(candidate_pairs_mi, df)
print(f"[Script con blocking combinato] Dimensioni della similarity_matrix: {similarity_matrix.shape}")


# 6) Definizione delle regole di matching (esempio)
matches = similarity_matrix[
    # (1) Nome simile 0.80 - 0.92
    ((similarity_matrix['name_sim'] > 0.80) & (similarity_matrix['name_sim'] < 0.92))
    |
    # (2) Nome > 0.9 e industry_sim > 0.5 e country_sim > 0.5
    ((similarity_matrix['name_sim'] > 0.9) & 
     (similarity_matrix['industry_sim'] > 0.5) &
     (similarity_matrix['country_sim'] > 0.5))
]

print(f"[Script con blocking combinato] Numero di coppie finali considerate 'match': {len(matches)}")

# 7) Recupero dei record (join con df originale)
matches = matches.reset_index()
matches.rename(columns={'level_0': 'id_left', 'level_1': 'id_right'}, inplace=True)
matches = matches.merge(df, left_on='id_left', right_index=True, how='left', suffixes=('', '_left'))
matches = matches.merge(df, left_on='id_right', right_index=True, how='left', suffixes=('_left', '_right'))

# Eliminiamo le righe in cui i campi company_name sono identici (stesso record)
matches = matches[matches['company_name_left'] != matches['company_name_right']]

# 8) Salvataggio in CSV
output_file = "matched_companies_detailed_blocking_2.csv"
matches.to_csv(output_file, index=False)
print(f"[Script con blocking combinato] ✅ File '{output_file}' generato!")


C:\Users\luca-\AppData\Local\Temp\ipykernel_24856\1170323690.py:6: DtypeWarning: Columns (2,5,6,7,9,14,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(schema_file)


[Script con blocking combinato] Numero di record iniziali: 75793
[Script con blocking combinato] Numero di coppie candidate: 46228525


In [ ]:
import pandas as pd

# Path dei file (aggiorna con i percorsi reali)
pairwise_file = 'matched_companies_detailed_blocking_2.csv'
groundtruth_file = 'data/ground_truth.csv'

# Caricamento dei file
pairwise = pd.read_csv(pairwise_file)
groundtruth = pd.read_csv(groundtruth_file)

# Funzione di supporto per normalizzare i nomi di società (rimuove spazi e usa lowercase)
def normalize_name(name):
    return name.strip().lower() if isinstance(name, str) else str(name).lower()

# Funzione per ottenere una tupla ordinata, così che l'ordine non conti
def normalize_pair(row, col1, col2):
    left = normalize_name(row[col1])
    right = normalize_name(row[col2])
    return tuple(sorted([left, right]))

# Creiamo la colonna 'normalized_pair' in entrambi i dataset
pairwise['normalized_pair'] = pairwise.apply(
    lambda x: normalize_pair(x, 'company_name_left', 'company_name_right'), axis=1
)
groundtruth['normalized_pair'] = groundtruth.apply(
    lambda x: normalize_pair(x, 'company_a', 'company_b'), axis=1
)

# Convertiamo in insiemi per confronto
pairwise_set = set(pairwise['normalized_pair'])
groundtruth_set = set(groundtruth['normalized_pair'])

# Intersezione tra le coppie del dataset pairwise e quelle della ground truth (tutte, indipendentemente dalla label)
intersection = pairwise_set.intersection(groundtruth_set)

# Filtriamo la ground truth per ottenere solo le coppie con label 1 (cioè i match veri)
groundtruth_true_set = set(groundtruth[groundtruth['label'] == 1]['normalized_pair'])

# Calcoliamo il numero di coppie dell'intersezione che hanno label 1
tp = len(intersection.intersection(groundtruth_true_set))

# Calcolo delle metriche:
# - Precisione: fra le coppie previste (cioè quelle in intersezione), quanti sono corretti (label=1)
precision = tp / len(intersection) if len(intersection) else 0
# - Recall: fra tutte le coppie corrette presenti in ground truth, quanti sono stati trovati
recall = tp / len(groundtruth_true_set) if len(groundtruth_true_set) else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

# Output dei risultati
print(f"Numero di coppie in pairwise: {len(pairwise_set)}")
print(f"Numero di coppie in ground truth: {len(groundtruth_set)}")
print(f'Numero di coppie in ground truth con label a 1: {len(groundtruth_true_set)}')
print(f"Numero di coppie in intersezione: {len(intersection)}")
print(f"Numero di coppie nell'intersezione con label 1: {tp}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")


Numero di coppie in pairwise: 82726
Numero di coppie in ground truth: 182
Numero di coppie in ground truth con label a 1: 82
Numero di coppie in intersezione: 167
Numero di coppie nell'intersezione con label 1: 73
Precision: 0.4371
Recall: 0.8902
F1-Score: 0.5863
